## 1. Setup & Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
import os

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Plotly template
TEMPLATE = 'plotly_white'
COLORS = px.colors.qualitative.Set2

# Create output directories
os.makedirs('../reports/csv', exist_ok=True)
os.makedirs('../reports/html', exist_ok=True)

print('✅ Libraries imported successfully!')
print(f'📅 Analysis Date: {datetime.now().strftime("%Y-%m-%d %H:%M")}')

In [ ]:
# Load all datasets
DATA_PATH = '../data/'

marketing_spend = pd.read_csv(f'{DATA_PATH}marketing_spend.csv', parse_dates=['date'])
funnel_events = pd.read_csv(f'{DATA_PATH}funnel_events.csv', parse_dates=['timestamp'])
revenue_marketing = pd.read_csv(f'{DATA_PATH}revenue_marketing.csv', parse_dates=['date'])
revenue_finance = pd.read_csv(f'{DATA_PATH}revenue_finance.csv', parse_dates=['date'])

# Add date column to funnel events
funnel_events['date'] = funnel_events['timestamp'].dt.date
funnel_events['date'] = pd.to_datetime(funnel_events['date'])

print('✅ All datasets loaded!')
print(f'   • Marketing Spend: {len(marketing_spend)} rows')
print(f'   • Funnel Events: {len(funnel_events)} rows')
print(f'   • Revenue (Marketing): {len(revenue_marketing)} rows')
print(f'   • Revenue (Finance): {len(revenue_finance)} rows')

---
## 2. True CAC by Channel (Finance Verified)

**CAC = Total Marketing Spend / Number of Verified Paid Users**

We calculate CAC using checkout events as the conversion point and allocate conversions to campaigns proportionally based on daily spend share.

In [ ]:
# Step 1: Calculate total spend per campaign
campaign_spend = marketing_spend.groupby('campaign').agg({
    'spend': 'sum',
    'date': ['min', 'max', 'count']
}).reset_index()
campaign_spend.columns = ['campaign', 'total_spend', 'start_date', 'end_date', 'active_days']

# Step 2: Count daily conversions (checkout events)
daily_conversions = funnel_events[funnel_events['event_type'] == 'checkout'].groupby('date').agg({
    'user_id': 'nunique'
}).reset_index()
daily_conversions.columns = ['date', 'conversions']

# Step 3: Calculate daily spend share per campaign
daily_total_spend = marketing_spend.groupby('date')['spend'].sum().reset_index()
daily_total_spend.columns = ['date', 'total_daily_spend']

spend_with_share = marketing_spend.merge(daily_total_spend, on='date')
spend_with_share['spend_share'] = spend_with_share['spend'] / spend_with_share['total_daily_spend']

# Step 4: Merge conversions and calculate attributed conversions
spend_conversions = spend_with_share.merge(daily_conversions, on='date', how='left')
spend_conversions['conversions'] = spend_conversions['conversions'].fillna(0)
spend_conversions['attributed_conversions'] = spend_conversions['conversions'] * spend_conversions['spend_share']

# Step 5: Aggregate by campaign for CAC calculation
cac_by_channel = spend_conversions.groupby('campaign').agg({
    'spend': 'sum',
    'attributed_conversions': 'sum'
}).reset_index()
cac_by_channel.columns = ['campaign', 'total_spend', 'attributed_conversions']

# Calculate CAC
cac_by_channel['cac'] = cac_by_channel['total_spend'] / cac_by_channel['attributed_conversions'].replace(0, np.nan)

# Add benchmark comparison (assuming $100 target CAC)
TARGET_CAC = 100
cac_by_channel['cac_status'] = cac_by_channel['cac'].apply(
    lambda x: 'Excellent' if x <= TARGET_CAC else ('Acceptable' if x <= TARGET_CAC * 1.5 else 'High')
)
cac_by_channel['vs_target'] = ((cac_by_channel['cac'] - TARGET_CAC) / TARGET_CAC * 100).round(1)

print('📊 TRUE CAC BY CHANNEL (Finance Verified)\n')
cac_by_channel

In [ ]:
# Interactive CAC Visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('CAC by Channel', 'CAC vs Target Benchmark'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# CAC by channel bar chart
colors_cac = ['#28a745' if s == 'Excellent' else '#ffc107' if s == 'Acceptable' else '#dc3545' 
              for s in cac_by_channel['cac_status']]

fig.add_trace(
    go.Bar(
        x=cac_by_channel['campaign'],
        y=cac_by_channel['cac'],
        marker_color=colors_cac,
        text=cac_by_channel['cac'].round(2),
        textposition='outside',
        name='CAC ($)',
        hovertemplate='<b>%{x}</b><br>CAC: $%{y:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Add target line
fig.add_hline(y=TARGET_CAC, line_dash='dash', line_color='red', 
              annotation_text=f'Target: ${TARGET_CAC}', row=1, col=1)

# Variance vs target
fig.add_trace(
    go.Bar(
        x=cac_by_channel['campaign'],
        y=cac_by_channel['vs_target'],
        marker_color=colors_cac,
        text=cac_by_channel['vs_target'].apply(lambda x: f"{x:+.1f}%"),
        textposition='outside',
        name='% vs Target',
        hovertemplate='<b>%{x}</b><br>vs Target: %{y:+.1f}%<extra></extra>'
    ),
    row=1, col=2
)

fig.add_hline(y=0, line_color='black', line_width=1, row=1, col=2)

fig.update_layout(
    title_text='<b>Customer Acquisition Cost (CAC) Analysis</b>',
    template=TEMPLATE,
    showlegend=False,
    height=450
)

fig.update_yaxes(title_text='CAC ($)', row=1, col=1)
fig.update_yaxes(title_text='% Variance from Target', row=1, col=2)

fig.show()

# Export to HTML
fig.write_html('../reports/html/cac_analysis.html')

---
## 3. ROAS Comparison: Marketing vs Finance

**ROAS = Revenue / Marketing Spend**

We compare:
- **Marketing ROAS**: Using marketing-reported revenue
- **Finance ROAS**: Using finance-verified revenue (allocated proportionally)

In [ ]:
# Calculate ROAS by campaign

# Marketing Revenue by campaign
mkt_revenue_by_campaign = revenue_marketing.groupby('campaign')['revenue'].sum().reset_index()
mkt_revenue_by_campaign.columns = ['campaign', 'marketing_revenue']

# Total Finance Revenue (we'll allocate proportionally by spend)
total_finance_revenue = revenue_finance['revenue'].sum()
total_spend = marketing_spend['spend'].sum()

# Create ROAS comparison table
roas_comparison = campaign_spend[['campaign', 'total_spend']].copy()
roas_comparison = roas_comparison.merge(mkt_revenue_by_campaign, on='campaign', how='left')
roas_comparison['marketing_revenue'] = roas_comparison['marketing_revenue'].fillna(0)

# Allocate finance revenue proportionally by spend share
roas_comparison['spend_share'] = roas_comparison['total_spend'] / roas_comparison['total_spend'].sum()
roas_comparison['finance_revenue'] = roas_comparison['spend_share'] * total_finance_revenue

# Calculate ROAS
roas_comparison['marketing_roas'] = roas_comparison['marketing_revenue'] / roas_comparison['total_spend']
roas_comparison['finance_roas'] = roas_comparison['finance_revenue'] / roas_comparison['total_spend']
roas_comparison['roas_gap'] = roas_comparison['marketing_roas'] - roas_comparison['finance_roas']
roas_comparison['roas_gap_pct'] = (roas_comparison['roas_gap'] / roas_comparison['finance_roas'] * 100).round(1)

# Profit/Loss
roas_comparison['mkt_profit'] = roas_comparison['marketing_revenue'] - roas_comparison['total_spend']
roas_comparison['fin_profit'] = roas_comparison['finance_revenue'] - roas_comparison['total_spend']

print('📊 ROAS COMPARISON: MARKETING VS FINANCE\n')
roas_comparison[['campaign', 'total_spend', 'marketing_revenue', 'finance_revenue', 
                 'marketing_roas', 'finance_roas', 'roas_gap']]

In [ ]:
# Interactive ROAS Comparison Chart
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'ROAS by Channel: Marketing vs Finance',
        'Revenue Comparison',
        'ROAS Gap Analysis',
        'Profit/Loss Comparison'
    ),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# 1. ROAS Comparison (grouped bar)
fig.add_trace(
    go.Bar(name='Marketing ROAS', x=roas_comparison['campaign'], 
           y=roas_comparison['marketing_roas'],
           marker_color='#0d6efd',
           text=roas_comparison['marketing_roas'].round(2).astype(str) + 'x',
           textposition='outside'),
    row=1, col=1
)
fig.add_trace(
    go.Bar(name='Finance ROAS', x=roas_comparison['campaign'], 
           y=roas_comparison['finance_roas'],
           marker_color='#28a745',
           text=roas_comparison['finance_roas'].round(2).astype(str) + 'x',
           textposition='outside'),
    row=1, col=1
)

# Add break-even line
fig.add_hline(y=1, line_dash='dash', line_color='red', 
              annotation_text='Break-even (1.0x)', row=1, col=1)

# 2. Revenue Comparison
fig.add_trace(
    go.Bar(name='Marketing Revenue', x=roas_comparison['campaign'], 
           y=roas_comparison['marketing_revenue'],
           marker_color='#0d6efd', showlegend=False),
    row=1, col=2
)
fig.add_trace(
    go.Bar(name='Finance Revenue', x=roas_comparison['campaign'], 
           y=roas_comparison['finance_revenue'],
           marker_color='#28a745', showlegend=False),
    row=1, col=2
)

# 3. ROAS Gap
gap_colors = ['#dc3545' if g > 0 else '#28a745' for g in roas_comparison['roas_gap']]
fig.add_trace(
    go.Bar(name='ROAS Gap', x=roas_comparison['campaign'], 
           y=roas_comparison['roas_gap'],
           marker_color=gap_colors,
           text=roas_comparison['roas_gap'].round(2).astype(str) + 'x',
           textposition='outside', showlegend=False),
    row=2, col=1
)
fig.add_hline(y=0, line_color='black', row=2, col=1)

# 4. Profit/Loss
fig.add_trace(
    go.Bar(name='Marketing P/L', x=roas_comparison['campaign'], 
           y=roas_comparison['mkt_profit'],
           marker_color='#0d6efd', showlegend=False),
    row=2, col=2
)
fig.add_trace(
    go.Bar(name='Finance P/L', x=roas_comparison['campaign'], 
           y=roas_comparison['fin_profit'],
           marker_color='#28a745', showlegend=False),
    row=2, col=2
)
fig.add_hline(y=0, line_color='black', row=2, col=2)

fig.update_layout(
    title_text='<b>ROAS Analysis: Marketing vs Finance Comparison</b>',
    template=TEMPLATE,
    height=700,
    barmode='group',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.update_yaxes(title_text='ROAS (x)', row=1, col=1)
fig.update_yaxes(title_text='Revenue ($)', row=1, col=2)
fig.update_yaxes(title_text='ROAS Gap', row=2, col=1)
fig.update_yaxes(title_text='Profit/Loss ($)', row=2, col=2)

fig.show()

# Export
fig.write_html('../reports/html/roas_comparison.html')

---
## 4. Variance Analysis (Daily Level)

Analyzes the discrepancy between Marketing-reported and Finance-verified revenue at the daily level.

In [ ]:
# Daily revenue aggregation
mkt_daily = revenue_marketing.groupby('date')['revenue'].sum().reset_index()
mkt_daily.columns = ['date', 'marketing_revenue']

fin_daily = revenue_finance.groupby('date')['revenue'].sum().reset_index()
fin_daily.columns = ['date', 'finance_revenue']

# Merge for comparison
daily_variance = pd.merge(mkt_daily, fin_daily, on='date', how='outer').fillna(0)
daily_variance = daily_variance.sort_values('date')

# Calculate variance metrics
daily_variance['variance'] = daily_variance['marketing_revenue'] - daily_variance['finance_revenue']
daily_variance['variance_pct'] = np.where(
    daily_variance['finance_revenue'] > 0,
    (daily_variance['variance'] / daily_variance['finance_revenue'] * 100).round(1),
    np.where(daily_variance['marketing_revenue'] > 0, 100, 0)
)
daily_variance['cumulative_variance'] = daily_variance['variance'].cumsum()

# Categorize variance
def categorize_variance(row):
    if row['marketing_revenue'] == 0 and row['finance_revenue'] > 0:
        return 'Finance Only'
    elif row['finance_revenue'] == 0 and row['marketing_revenue'] > 0:
        return 'Marketing Only'
    elif row['variance'] > 0:
        return 'Over-reported'
    elif row['variance'] < 0:
        return 'Under-reported'
    else:
        return 'Matched'

daily_variance['category'] = daily_variance.apply(categorize_variance, axis=1)

print('📊 DAILY VARIANCE ANALYSIS\n')
print(f"Total Marketing Revenue: ${daily_variance['marketing_revenue'].sum():,.0f}")
print(f"Total Finance Revenue:   ${daily_variance['finance_revenue'].sum():,.0f}")
print(f"Total Variance:          ${daily_variance['variance'].sum():,.0f}")
print(f"Variance %:              {daily_variance['variance'].sum() / daily_variance['finance_revenue'].sum() * 100:.1f}%")
print('\n')
daily_variance.head(10)

In [ ]:
# Interactive Variance Visualization
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        'Daily Revenue: Marketing vs Finance',
        'Daily Variance (Marketing - Finance)',
        'Cumulative Variance Over Time'
    ),
    vertical_spacing=0.1,
    row_heights=[0.35, 0.35, 0.3]
)

# 1. Revenue comparison line chart
fig.add_trace(
    go.Scatter(x=daily_variance['date'], y=daily_variance['marketing_revenue'],
               name='Marketing', line=dict(color='#0d6efd', width=2),
               mode='lines+markers'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=daily_variance['date'], y=daily_variance['finance_revenue'],
               name='Finance', line=dict(color='#28a745', width=2),
               mode='lines+markers'),
    row=1, col=1
)

# 2. Variance bar chart
variance_colors = ['#dc3545' if v > 0 else '#28a745' for v in daily_variance['variance']]
fig.add_trace(
    go.Bar(x=daily_variance['date'], y=daily_variance['variance'],
           name='Variance', marker_color=variance_colors,
           hovertemplate='Date: %{x}<br>Variance: $%{y:,.0f}<extra></extra>'),
    row=2, col=1
)
fig.add_hline(y=0, line_color='black', row=2, col=1)

# 3. Cumulative variance
fig.add_trace(
    go.Scatter(x=daily_variance['date'], y=daily_variance['cumulative_variance'],
               name='Cumulative', fill='tozeroy',
               line=dict(color='#dc3545', width=2),
               fillcolor='rgba(220, 53, 69, 0.3)'),
    row=3, col=1
)

fig.update_layout(
    title_text='<b>Revenue Variance Analysis: Marketing vs Finance</b>',
    template=TEMPLATE,
    height=800,
    showlegend=True,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.update_yaxes(title_text='Revenue ($)', row=1, col=1)
fig.update_yaxes(title_text='Variance ($)', row=2, col=1)
fig.update_yaxes(title_text='Cumulative ($)', row=3, col=1)

fig.show()

# Export
fig.write_html('../reports/html/variance_analysis.html')

In [ ]:
# Variance Summary by Category
variance_summary = daily_variance.groupby('category').agg({
    'date': 'count',
    'variance': 'sum',
    'marketing_revenue': 'sum',
    'finance_revenue': 'sum'
}).reset_index()
variance_summary.columns = ['Category', 'Days', 'Total Variance', 'Marketing Revenue', 'Finance Revenue']

# Pie chart for variance categories
fig = px.pie(
    variance_summary,
    values='Days',
    names='Category',
    title='<b>Variance Category Distribution</b>',
    color='Category',
    color_discrete_map={
        'Over-reported': '#dc3545',
        'Under-reported': '#28a745',
        'Matched': '#6c757d',
        'Marketing Only': '#fd7e14',
        'Finance Only': '#17a2b8'
    },
    template=TEMPLATE
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print('\n📊 Variance Summary by Category:')
variance_summary

---
## 5. Funnel Conversion Rates

Calculates stage-by-stage conversion rates through the marketing funnel.

In [ ]:
# Define funnel stages in order
FUNNEL_ORDER = ['page_view', 'add_to_cart', 'checkout', 'purchase']

# Count unique users at each stage
funnel_counts = funnel_events.groupby('event_type')['user_id'].nunique().reset_index()
funnel_counts.columns = ['stage', 'users']

# Filter and order
funnel_counts = funnel_counts[funnel_counts['stage'].isin(FUNNEL_ORDER)]
funnel_counts['stage_order'] = funnel_counts['stage'].apply(lambda x: FUNNEL_ORDER.index(x))
funnel_counts = funnel_counts.sort_values('stage_order')

# Calculate metrics
top_funnel = funnel_counts['users'].iloc[0]
funnel_counts['pct_of_top'] = (funnel_counts['users'] / top_funnel * 100).round(1)
funnel_counts['prev_users'] = funnel_counts['users'].shift(1)
funnel_counts['stage_conversion'] = (funnel_counts['users'] / funnel_counts['prev_users'] * 100).round(1)
funnel_counts['drop_off'] = funnel_counts['prev_users'] - funnel_counts['users']
funnel_counts['drop_off_rate'] = (funnel_counts['drop_off'] / funnel_counts['prev_users'] * 100).round(1)

# Clean up labels
funnel_counts['stage_label'] = funnel_counts['stage'].str.replace('_', ' ').str.title()

print('📊 FUNNEL CONVERSION RATES\n')
funnel_counts[['stage_label', 'users', 'pct_of_top', 'stage_conversion', 'drop_off_rate']]

In [ ]:
# Interactive Funnel Visualization
fig = go.Figure()

# Funnel chart
fig.add_trace(go.Funnel(
    y=funnel_counts['stage_label'],
    x=funnel_counts['users'],
    textposition='inside',
    textinfo='value+percent initial',
    opacity=0.85,
    marker=dict(
        color=['#0d6efd', '#6610f2', '#fd7e14', '#28a745'],
        line=dict(width=2, color='white')
    ),
    connector=dict(line=dict(color='#dee2e6', width=2))
))

fig.update_layout(
    title_text='<b>Marketing Funnel Conversion</b>',
    template=TEMPLATE,
    height=500
)

fig.show()

# Export
fig.write_html('../reports/html/funnel_analysis.html')

In [ ]:
# Stage-to-Stage Conversion Visualization
conversion_data = funnel_counts[funnel_counts['stage_conversion'].notna()].copy()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Stage-to-Stage Conversion Rate', 'Drop-off Rate by Stage'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Conversion rates
fig.add_trace(
    go.Bar(
        x=conversion_data['stage_label'],
        y=conversion_data['stage_conversion'],
        marker_color='#28a745',
        text=conversion_data['stage_conversion'].astype(str) + '%',
        textposition='outside',
        name='Conversion Rate'
    ),
    row=1, col=1
)

# Drop-off rates
dropoff_colors = ['#ffc107' if d < 30 else '#dc3545' for d in conversion_data['drop_off_rate']]
fig.add_trace(
    go.Bar(
        x=conversion_data['stage_label'],
        y=conversion_data['drop_off_rate'],
        marker_color=dropoff_colors,
        text=conversion_data['drop_off_rate'].astype(str) + '%',
        textposition='outside',
        name='Drop-off Rate'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='<b>Funnel Conversion & Drop-off Analysis</b>',
    template=TEMPLATE,
    height=400,
    showlegend=False
)

fig.update_yaxes(title_text='Conversion Rate (%)', range=[0, 110], row=1, col=1)
fig.update_yaxes(title_text='Drop-off Rate (%)', range=[0, 60], row=1, col=2)

fig.show()

---
## 6. Channel Performance Scorecard

Comprehensive scorecard combining all metrics for each marketing channel.

In [ ]:
# Build comprehensive scorecard
scorecard = campaign_spend[['campaign', 'total_spend', 'active_days']].copy()

# Add CAC metrics
scorecard = scorecard.merge(
    cac_by_channel[['campaign', 'attributed_conversions', 'cac', 'cac_status']], 
    on='campaign', how='left'
)

# Add ROAS metrics
scorecard = scorecard.merge(
    roas_comparison[['campaign', 'marketing_revenue', 'finance_revenue', 
                     'marketing_roas', 'finance_roas', 'roas_gap']], 
    on='campaign', how='left'
)

# Calculate efficiency score (composite metric)
# Normalize metrics (0-100 scale)
scorecard['cac_score'] = 100 - (scorecard['cac'] / scorecard['cac'].max() * 100)
scorecard['roas_score'] = scorecard['finance_roas'] / scorecard['finance_roas'].max() * 100
scorecard['efficiency_score'] = ((scorecard['cac_score'] + scorecard['roas_score']) / 2).round(1)

# Performance tier
def assign_tier(score):
    if score >= 80:
        return 'Tier 1 - Star'
    elif score >= 60:
        return 'Tier 2 - Good'
    elif score >= 40:
        return 'Tier 3 - Average'
    else:
        return 'Tier 4 - Underperform'

scorecard['performance_tier'] = scorecard['efficiency_score'].apply(assign_tier)

# Budget recommendation
def budget_rec(row):
    if row['finance_roas'] >= 1.5 and row['cac'] <= 120:
        return 'INCREASE'
    elif row['finance_roas'] >= 1.0:
        return 'MAINTAIN'
    else:
        return 'REDUCE'

scorecard['budget_recommendation'] = scorecard.apply(budget_rec, axis=1)

print('📊 CHANNEL PERFORMANCE SCORECARD\n')
scorecard[['campaign', 'total_spend', 'attributed_conversions', 'cac', 
           'finance_roas', 'efficiency_score', 'performance_tier', 'budget_recommendation']]

In [ ]:
# Interactive Scorecard Visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Efficiency Score by Channel',
        'CAC vs ROAS Scatter',
        'Spend vs Revenue Bubble',
        'Performance Tier Distribution'
    ),
    specs=[[{'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'pie'}]]
)

# 1. Efficiency Score
tier_colors = {
    'Tier 1 - Star': '#28a745',
    'Tier 2 - Good': '#17a2b8',
    'Tier 3 - Average': '#ffc107',
    'Tier 4 - Underperform': '#dc3545'
}
colors_eff = [tier_colors.get(t, '#6c757d') for t in scorecard['performance_tier']]

fig.add_trace(
    go.Bar(
        x=scorecard['campaign'],
        y=scorecard['efficiency_score'],
        marker_color=colors_eff,
        text=scorecard['efficiency_score'],
        textposition='outside',
        name='Efficiency'
    ),
    row=1, col=1
)

# 2. CAC vs ROAS Scatter
fig.add_trace(
    go.Scatter(
        x=scorecard['cac'],
        y=scorecard['finance_roas'],
        mode='markers+text',
        marker=dict(size=20, color=colors_eff),
        text=scorecard['campaign'],
        textposition='top center',
        name='CAC vs ROAS',
        hovertemplate='<b>%{text}</b><br>CAC: $%{x:.0f}<br>ROAS: %{y:.2f}x<extra></extra>'
    ),
    row=1, col=2
)

# Add quadrant lines
fig.add_vline(x=TARGET_CAC, line_dash='dash', line_color='gray', row=1, col=2)
fig.add_hline(y=1, line_dash='dash', line_color='gray', row=1, col=2)

# 3. Spend vs Revenue Bubble
fig.add_trace(
    go.Scatter(
        x=scorecard['total_spend'],
        y=scorecard['finance_revenue'],
        mode='markers+text',
        marker=dict(
            size=scorecard['attributed_conversions'] * 2 + 10,
            color=colors_eff,
            opacity=0.7
        ),
        text=scorecard['campaign'],
        textposition='top center',
        name='Spend vs Revenue',
        hovertemplate='<b>%{text}</b><br>Spend: $%{x:,.0f}<br>Revenue: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# Add break-even line (revenue = spend)
max_val = max(scorecard['total_spend'].max(), scorecard['finance_revenue'].max())
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Break-even',
        showlegend=False
    ),
    row=2, col=1
)

# 4. Tier Distribution Pie
tier_dist = scorecard['performance_tier'].value_counts().reset_index()
tier_dist.columns = ['tier', 'count']
tier_dist['color'] = tier_dist['tier'].map(tier_colors)

fig.add_trace(
    go.Pie(
        labels=tier_dist['tier'],
        values=tier_dist['count'],
        marker_colors=tier_dist['color'],
        textinfo='label+percent',
        hole=0.4,
        name='Tier Distribution'
    ),
    row=2, col=2
)

fig.update_layout(
    title_text='<b>Channel Performance Scorecard Dashboard</b>',
    template=TEMPLATE,
    height=800,
    showlegend=False
)

fig.update_yaxes(title_text='Score', range=[0, 100], row=1, col=1)
fig.update_xaxes(title_text='CAC ($)', row=1, col=2)
fig.update_yaxes(title_text='ROAS (x)', row=1, col=2)
fig.update_xaxes(title_text='Total Spend ($)', row=2, col=1)
fig.update_yaxes(title_text='Finance Revenue ($)', row=2, col=1)

fig.show()

# Export
fig.write_html('../reports/html/scorecard_dashboard.html')

---
## 7. Export Results to CSV

In [ ]:
# Export all analysis results to CSV
OUTPUT_PATH = '../reports/csv/'

# 1. CAC Analysis
cac_by_channel.to_csv(f'{OUTPUT_PATH}cac_by_channel.csv', index=False)
print('✅ Exported: cac_by_channel.csv')

# 2. ROAS Comparison
roas_comparison.to_csv(f'{OUTPUT_PATH}roas_comparison.csv', index=False)
print('✅ Exported: roas_comparison.csv')

# 3. Daily Variance
daily_variance.to_csv(f'{OUTPUT_PATH}daily_variance.csv', index=False)
print('✅ Exported: daily_variance.csv')

# 4. Funnel Metrics
funnel_export = funnel_counts[['stage_label', 'users', 'pct_of_top', 'stage_conversion', 'drop_off_rate']].copy()
funnel_export.to_csv(f'{OUTPUT_PATH}funnel_metrics.csv', index=False)
print('✅ Exported: funnel_metrics.csv')

# 5. Channel Scorecard
scorecard_export = scorecard[['campaign', 'total_spend', 'attributed_conversions', 'cac', 
                              'marketing_revenue', 'finance_revenue', 'marketing_roas', 
                              'finance_roas', 'efficiency_score', 'performance_tier', 
                              'budget_recommendation']].copy()
scorecard_export.to_csv(f'{OUTPUT_PATH}channel_scorecard.csv', index=False)
print('✅ Exported: channel_scorecard.csv')

# 6. Summary Metrics (single row)
summary_metrics = pd.DataFrame([{
    'total_marketing_spend': marketing_spend['spend'].sum(),
    'total_marketing_revenue': revenue_marketing['revenue'].sum(),
    'total_finance_revenue': revenue_finance['revenue'].sum(),
    'revenue_variance': revenue_marketing['revenue'].sum() - revenue_finance['revenue'].sum(),
    'revenue_variance_pct': (revenue_marketing['revenue'].sum() - revenue_finance['revenue'].sum()) / revenue_finance['revenue'].sum() * 100,
    'overall_marketing_roas': revenue_marketing['revenue'].sum() / marketing_spend['spend'].sum(),
    'overall_finance_roas': revenue_finance['revenue'].sum() / marketing_spend['spend'].sum(),
    'total_conversions': funnel_events[funnel_events['event_type'] == 'checkout']['user_id'].nunique(),
    'overall_cac': marketing_spend['spend'].sum() / funnel_events[funnel_events['event_type'] == 'checkout']['user_id'].nunique(),
    'analysis_date': datetime.now().strftime('%Y-%m-%d')
}])
summary_metrics.to_csv(f'{OUTPUT_PATH}summary_metrics.csv', index=False)
print('✅ Exported: summary_metrics.csv')

print('\n' + '='*50)
print('📁 All exports complete! Files saved to reports/csv/')

---
## 8. Executive Summary

In [ ]:
# Print executive summary
total_spend = marketing_spend['spend'].sum()
mkt_rev = revenue_marketing['revenue'].sum()
fin_rev = revenue_finance['revenue'].sum()
variance = mkt_rev - fin_rev
variance_pct = variance / fin_rev * 100

best_channel = scorecard.loc[scorecard['efficiency_score'].idxmax(), 'campaign']
worst_channel = scorecard.loc[scorecard['efficiency_score'].idxmin(), 'campaign']

biggest_bottleneck = funnel_counts.loc[funnel_counts['drop_off_rate'].idxmax()]

print('═' * 70)
print('                    EXECUTIVE SUMMARY                               ')
print('═' * 70)
print(f'''
📌 FINANCIAL OVERVIEW
   • Total Marketing Spend:      ${total_spend:>12,.0f}
   • Marketing Reported Revenue: ${mkt_rev:>12,.0f}
   • Finance Verified Revenue:   ${fin_rev:>12,.0f}
   • Revenue Variance:           ${variance:>12,.0f} ({variance_pct:.1f}%)

📌 ROAS PERFORMANCE
   • Marketing ROAS:             {mkt_rev/total_spend:.2f}x
   • Finance-Adjusted ROAS:      {fin_rev/total_spend:.2f}x
   • ROAS Gap:                   {(mkt_rev-fin_rev)/total_spend:.2f}x overstated

📌 CHANNEL INSIGHTS
   • Best Performing Channel:    {best_channel} (Score: {scorecard.loc[scorecard['campaign']==best_channel, 'efficiency_score'].values[0]:.1f})
   • Worst Performing Channel:   {worst_channel} (Score: {scorecard.loc[scorecard['campaign']==worst_channel, 'efficiency_score'].values[0]:.1f})
   • Average CAC:                ${scorecard['cac'].mean():,.0f}

📌 FUNNEL ANALYSIS
   • Overall Conversion Rate:    {funnel_counts.iloc[-1]['pct_of_top']:.1f}%
   • Biggest Bottleneck:         {biggest_bottleneck['stage_label']} ({biggest_bottleneck['drop_off_rate']:.1f}% drop-off)

═══════════════════════════════════════════════════════════════════════
⚠️  KEY FINDING: Marketing is over-reporting revenue by {variance_pct:.0f}%
    Recommend immediate audit of attribution methodology.
═══════════════════════════════════════════════════════════════════════
''')

---
## ✅ Analysis Complete

### Outputs Generated:

**Interactive HTML Charts:**
- `reports/html/cac_analysis.html`
- `reports/html/roas_comparison.html`
- `reports/html/variance_analysis.html`
- `reports/html/funnel_analysis.html`
- `reports/html/scorecard_dashboard.html`

**CSV Exports for Dashboard:**
- `reports/csv/cac_by_channel.csv`
- `reports/csv/roas_comparison.csv`
- `reports/csv/daily_variance.csv`
- `reports/csv/funnel_metrics.csv`
- `reports/csv/channel_scorecard.csv`
- `reports/csv/summary_metrics.csv`